# TORI WEB SCRAPER AND DATA ANALYZER V1.0

This is a script that will get prices from each gpu on tori and afterwards analyze them. 

## 1. Simple title and price scraping method

## Description fetcher


In [25]:
import pandas as pd
from IPython.display import display, HTML

# Read the CSV file into a DataFrame
df = pd.read_csv('gpu_cleaned_data.csv')

# Display the first 5 rows of the DataFrame in a pretty way
print("Head of DataFrame:")
#df.describe()
display(HTML(df.tail(60).to_html(index=False, classes='table table-striped table-hover')))

Head of DataFrame:


Title,Price,GPU_Category,VRAM,Description,Link
Näytönohjain strix-gtx960-dc2oc-2gd5,35.00,960,Unknown,"Lisätiedot\n\t\t\tASUS Strix GeForce GTX 960 tehtaan\nylikellotusytimellä tarjoaa viileän ja hiljaisen\npelikokemuksenOminaisuudet1291MHz tehtaalla ylikellotettu näytönohjaimen1317MHz näyttösuorittimen boostauskello OC-tilassaDirectCU II 0dB tuuletinteknologialla varustettuna\nantaa sinun nauttia kevyistä peleistä täydessä\nhiljaisuudessa ja vaativimmista peleistä 30 %\nviileämmin 3X hiljaisemmalla suorituskyvyllä!Koneen tehoa säätelevät premium alloy-osat\nviilentävät koneen osia, jotka siten toimivat 15%\nnopeammin ja kestävät 2,5 kertaa pidempään.GPU Tweak tarjoaa helpon käyttöliittymän, jonka\navulla voit muuttaa kellotaajuuksia, jännitteitä,\ntuulettimien nopeuksia ja paljon muuta.GPU Tweak XSplit Gamecasterilla: Reaaliaikainen ja\nintuitiivinen näyttösuorittimen kelloboostaus,\njolla voit ladata pelisi välittömästi",https://www.tori.fi/uusimaa/Naytonohjain_strix_gtx960_dc2oc_2gd5_113630809.htm?ca=18&w=1
ASUS GeForce RTX 3090 ROG Strix OC Edition,1370.00,3090,24gb,"Lisätiedot\n\t\t\tTäysin koskematon/käyttämätön, 2v takuu löytyy ja\nkuitti. Toinen ylimääräinen kappale jonka sain kaupasta\nnoin alle vuosi sitten ja ei ole ollut tarvetta\nomaan käyttöön vaikka suunnitelmia ollut. En ole avannut pakkausta ollenkaan, vain\npölyttynyt nurkassa muovit päällä. Voi kysyä lisätietoja viesteillä rohkeasti. Onnistuu joko nouto tai lähetys. CUDA coret: 10496 Kellotaajuus: OC Mode: 1890 MHz (Boost Clock) Gaming Mode: 1860 MHz (Boost Clock) Muisti Määrä: 24GB GDDR6X Muistiväylä: 384-bit Kellotajuus: 19.5 Gbps Liitännät 2 x HDMI 2.1 3 x DisplayPort 1.4a Väylä: PCI-E 4.0 x16 Mitat: 318,5 x 140,1 x 57,78 mm / 2.9 slot Paino: 1770g Suositeltu virtalähteen koko: 850W Virtaliitännät: 3 x 8-pin NVLink -tuki: Kyllä",https://www.tori.fi/uusimaa/ASUS_GeForce_RTX_3090_ROG_Strix_OC_Edition_107091673.htm?ca=18&w=1
MSI GeForce Gaming X GTX 1070 8GB,130.00,1070,8gb,Lisätiedot\n\t\t\tHyväkuntoinen ja toimiva näytönohjain\npelikäyttöön:MSI GeForce Gaming X GTX 1070 8GB,https://www.tori.fi/uusimaa/MSI_GeForce_Gaming_X_GTX_1070_8GB_111026859.htm?ca=18&w=1
ZOTAC GAMING GeForce RTX 3060 Ti Twin Edge,250.00,3060,Unknown,"Lisätiedot\n\t\t\tMyydään ZOTAC GAMING GeForce RTX 3060 Ti Twin Edge\nnäytönohjain toimii moitteetta.Alkuperäistä pakkausta ei ole tallessa, mutta\npostitusta varten löytyy kyllä vastaava tukeva\npahvilaatikko ja runsaasti pehmikkeitä.3 x DisplayPort 1.4a 1 x HDMI 2.1 Nouto Vantaalta. Voin myös tuoda näytönohjaimen\npk-seudulle lisäkorvausta vastaan.Maksu mobilepay,käteisellä tai tilisiirto. hakusanat #3060ti #geforce #rtx3060ti #zotac\n#naytonohjain #näyttis",https://www.tori.fi/uusimaa/ZOTAC_GAMING_GeForce_RTX_3060_Ti_Twin_Edge_111020475.htm?ca=18&w=1
Gigabyte RTX 2080 vesijäähdytyksellä,320.00,2080,Unknown,"Lisätiedot\n\t\t\tVarattu toistaiseksi!!!Gigabyte GeForce RTX 2080 GAMING OC 8G 8192 Mt\n-näytönohjain PCI-e-väyläänMukana vesijäähdytysKäytetty vain pelaamiseen, ei louhittu.Täysin toimiva.Mieluiten nouto, mutta jos haluat pakettina niin\nhuomioi että vesijäähdytyksen takia vaatii ison\npaketin ja se maksaa n.13,90",https://www.tori.fi/uusimaa/Gigabyte_RTX_2080_vesijaahdytyksella_109520140.htm?ca=18&w=1
"Näytönohjain: ASUS GeForce GTX 470 Fermi, 1 GB",35.00,470,1 gb,Lisätiedot\n\t\t\tHyvin toimiva osa pois päivityksen altahttps://www.techpowerup.com/review/asus-geforce-gtx-470/2.html,https://www.tori.fi/uusimaa/Naytonohjain__ASUS_GeForce_GTX_470_Fermi__1_GB_106987286.htm?ca=18&w=1
"Näytönohjain: Gigabyte GTX 670 OC, 2 GB",37.00,Unknown,2 gb,Lisätiedot\n\t\t\tIhan kivasti toimiva osahttps://www.techpowerup.com/review/gigabyte-geforce-gtx-670-windforce/3.html,https://www.tori.fi/uusimaa/Naytonohjain__Gigabyte_GTX_670_OC__2_GB_106987068.htm?ca=18&w=1&last=1
Title not found,Price not found,Unknown,Unknown,Description not found,https://www.tori.fi/uusimaa/Gigabyte_RTX_3060_Ti_Eagle_8GB__LHR_118818362.htm?ca=18&w=1\n
GTG-1070 8Gb näytönoh